**Hello,**

**In this kernel, I will try maybe some kind of weird thing. Firstly I will imply a tree based model and will use the output of this model as it was a column in the dataset and try to implement a basic neural network.**

**I am hoping to see that this strategy will give me an acceptable outcome**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
df_train = pd.read_csv("../input/train.csv")
df_test = pd.read_csv("../input/test.csv")
df = pd.concat([df_train, df_test],axis=0, sort='False', ignore_index = True)

In [ ]:
df = df[df.columns.difference(['Id'])]

In [ ]:
df_train.head()

In [ ]:
df_test["Id"].head()

In [ ]:
ids = df_test["Id"]

Percantage of the empty values for each column is shown above. I feel like we can fill na's with zeros instead of dropping them

In [ ]:
df = df.fillna(0)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Flatten

In [ ]:
#Encoding categorical data
df = pd.get_dummies(df)

In [ ]:
print("Shape of our dataset is {}".format(df.shape))

# Lenght of train is 1460

Now the current dataframe is like:

In [ ]:
df.tail(3)

In [ ]:
df_train = df.iloc[:1460,:]
df_test = df.iloc[1460:,:]
X_train = df_train[df_train.columns.difference(['SalePrice'])].values
y_train = df_train[['SalePrice']].values
X_test = df_test[df_test.columns.difference(['SalePrice'])].values

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler, PowerTransformer
pt_X = PowerTransformer(method='yeo-johnson', standardize=False)
sc_y = StandardScaler()
sc_X = StandardScaler()
y_train = sc_y.fit_transform(y_train)
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)


In [ ]:
y_train[:5]

In [ ]:
y_t = y_train.flatten()
y_t.shape

In [ ]:
import lightgbm as lgb
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_t)
# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=250)
# predict
lgbm_prediction_tr = gbm.predict(X_train, num_iteration=gbm.best_iteration)
lgbm_prediction_te = gbm.predict(X_test, num_iteration=gbm.best_iteration)


Now the result of lightgbm will be added to dataset

In [ ]:
df_train["lgb"] = lgbm_prediction_tr
df_test["lgb"] = lgbm_prediction_te

In [ ]:
df_train.head()

SO, let's repeat it all

In [ ]:
df = pd.concat([df_train, df_test],axis=0, sort='False', ignore_index = True)
df = df[df.columns.difference(['Id'])]
df_train = df.iloc[:1460,:]
df_test = df.iloc[1460:,:]
X_train = df_train[df_train.columns.difference(['SalePrice'])].values
y_train = df_train[['SalePrice']].values
X_test = df_test[df_test.columns.difference(['SalePrice'])].values
pt_X = PowerTransformer(method='yeo-johnson', standardize=False)
sc_y = StandardScaler()
sc_X = StandardScaler()
y_train = sc_y.fit_transform(y_train)
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.callbacks import ModelCheckpoint

I couldnt find a way to hide output of the cell below.

In [ ]:
#Inıtialising the ANN
model = Sequential()
#Adding the input layer and first hidden layer
model.add(Dense(units =480, kernel_initializer='random_uniform', activation= 'tanh', 
                input_dim=X_train.shape[1]))
#Add the second hidden layer
model.add(Dense(units =480, kernel_initializer='random_uniform', activation= 'tanh'))
#Add the second hidden layer

model.add(Dense(units =10, kernel_initializer='random_uniform', activation= 'relu'))
#The output layer
model.add(Dense(units =1, kernel_initializer='random_uniform', activation= 'elu'))

#Compiling the ANN
opt = keras.optimizers.Adam(lr=0.0015, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=opt, loss='mean_squared_logarithmic_error', metrics=['mse'])
#Fitting the ANN to the training set
model_filepath = 'min_vl_model.h5'
checkpoint = ModelCheckpoint(model_filepath, monitor = 'val_loss', verbose=1, save_best_only = True, mode='min' )
model.fit(X_train,y_train, validation_split=0.07, batch_size=32, nb_epoch=3000, callbacks=[checkpoint])
model.load_weights(model_filepath)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred = sc_y.inverse_transform(y_pred)

In [ ]:
y_pred

In [ ]:
y_pred = pd.DataFrame(y_pred)
y_pred["Id"] = ids

In [ ]:
y_pred = y_pred.rename(columns={0: "SalePrice"})
y_pred = y_pred[["Id","SalePrice"]]
y_pred.to_csv("Submission.csv", index=False)

In [ ]:
y_pred.head()